In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import pandas as pd
import tensorflow as t

In [26]:
# Import our input dataset
ranking_df = pd.read_csv("ranking.csv")
ranking_df.head()

,TEAM_ID,LEAGUE_ID,SEASON_ID,STANDINGSDATE,CONFERENCE,TEAM,G,W,L,W_PCT,HOME_RECORD,ROAD_RECORD,RETURNTOPLAY
0,1610612762,0,22020,2021-03-21,West,Utah,41,30,11,0.732,16-2,14-9,NaN
1,1610612756,0,22020,2021-03-21,West,Phoenix,41,28,13,0.683,15-8,13-5,NaN
2,1610612747,0,22020,2021-03-21,West,L.A. Lakers,43,28,15,0.651,14-9,14-6,NaN
3,1610612746,0,22020,2021-03-21,West,LA Clippers,43,27,16,0.628,14-6,13-10,NaN
4,1610612743,0,22020,2021-03-21,West,Denver,42,25,17,0.595,12-9,13-8,NaN


In [28]:
# Generate our categorical variable list
ranking_cat = ranking_df.dtypes[ranking_df.dtypes == "object"].index.tolist()


# Check the number of unique values in each column
ranking_df[ranking_cat].nunique()

STANDINGSDATE    6382
CONFERENCE          2
TEAM               34
HOME_RECORD       800
ROAD_RECORD       752
dtype: int64

In [29]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
basketball_df = pd.DataFrame(enc.fit_transform(games_df[games_cat]))

# Add the encoded variable names to the dataframe
basketball_df.columns = enc.get_feature_names(games_cat)
basketball_df.head()

,GAME_DATE_EST_2003-10-05,GAME_DATE_EST_2003-10-06,GAME_DATE_EST_2003-10-07,GAME_DATE_EST_2003-10-08,GAME_DATE_EST_2003-10-09,GAME_DATE_EST_2003-10-10,GAME_DATE_EST_2003-10-11,GAME_DATE_EST_2003-10-12,GAME_DATE_EST_2003-10-13,GAME_DATE_EST_2003-10-14,...,GAME_DATE_EST_2021-03-13,GAME_DATE_EST_2021-03-14,GAME_DATE_EST_2021-03-15,GAME_DATE_EST_2021-03-16,GAME_DATE_EST_2021-03-17,GAME_DATE_EST_2021-03-18,GAME_DATE_EST_2021-03-19,GAME_DATE_EST_2021-03-20,GAME_DATE_EST_2021-03-21,GAME_STATUS_TEXT_Final
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
